In [1]:
! pip install psycopg2-binary --user
! pip install --upgrade pip

import pandas as pd
import psycopg2
import numpy as np
from getpass import getpass

You are using pip version 10.0.1, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 1.5MB 20.4MB/s ta 0:00:01
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


In [2]:
# connect to database
connection = psycopg2.connect(
    database  = "postgres",
    user      = "postgres",
    password  = getpass(),
    host      = "movie-rec-scrape.cvslmiksgnix.us-east-1.rds.amazonaws.com",
    port      = '5432'
)


 ················


In [3]:
# cursor object
try:
    c = connection.cursor()
    print("Connected!")
except:
    print("Connection problem chief!")

Connected!


In [4]:
c.execute("SELECT movie_id, username FROM reviews")

In [5]:
result = c.fetchall()
result

[('0233469', 'maxmik'),
 ('0233469', 'rzajac'),
 ('0233469', '=G='),
 ('0233469', 'toucanbob'),
 ('0233469', 'Spleen'),
 ('0233469', 'sudipta-maji'),
 ('0233469', 'uds3'),
 ('0233469', 'hydrocilator'),
 ('0233469', 'flipgirl38'),
 ('0233469', 'dsensing-1'),
 ('0233469', 'martyfin'),
 ('0233469', 'yadaks'),
 ('0233469', 'donald_willy'),
 ('0233469', 'elmonose'),
 ('0233469', 'blodhemn'),
 ('0233469', 'jncarr'),
 ('0234105', 'ochiru tenshi'),
 ('0233469', 'dentedfxp'),
 ('0233469', 'gazza-21'),
 ('0233469', 'Jbbarger'),
 ('0233469', 'martman99'),
 ('0233469', 'pugsly'),
 ('0233469', 'xenosis'),
 ('0233469', 'Norrin Radd'),
 ('0233469', 'Merely'),
 ('0233469', 'emorr'),
 ('0233469', 'george.schmidt'),
 ('0234105', 'scott_thai'),
 ('0233469', 'JACKAL-22'),
 ('0233469', 'Radio_Lady'),
 ('0233469', 'o0supergeek0o'),
 ('0233469', 'dszwerin'),
 ('0233469', 'steveefford'),
 ('0233469', 'armyson14'),
 ('0233469', 'mowglisj'),
 ('0233469', 'MovieCrap'),
 ('0234112', 'dbdumonteil'),
 ('0233469', '

In [6]:
df = pd.DataFrame(result, columns = ['movieid', 'userid'])
df.head()

,movieid,userid
0,0233469,maxmik
1,0233469,rzajac
2,0233469,=G=
3,0233469,toucanbob
4,0233469,Spleen


In [7]:
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'

In [8]:
df2 = pd.read_csv(url, sep='\t')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df2['movieid'] =[i[2:] for i in df2['tconst']]

In [10]:
df2['movieid'] = df2['movieid'].astype(int)

In [11]:
df2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,movieid
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",2
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",3
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",4
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",5


In [14]:
df.movieid = df.movieid.astype(int)

In [15]:
df = df.merge(df2, how='inner', on='movieid')

In [16]:
df.head()

,movieid,userid,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,233469,maxmik,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller"
1,233469,rzajac,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller"
2,233469,=G=,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller"
3,233469,toucanbob,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller"
4,233469,Spleen,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller"


In [17]:
df.movieid.value_counts()

7286456    8770
4154796    8462
111161     7158
4154664    7044
2527336    6495
468569     6441
2527338    6131
120737     5216
2488496    4663
4154756    4144
133093     4056
7131622    3955
1477834    3740
120915     3724
816692     3634
121765     3577
2975590    3576
167260     3536
1270797    3526
137523     3504
1375666    3479
185937     3450
121766     3441
499549     3324
1727824    3268
1825683    3096
480249     2992
1345836    2899
7784604    2871
372784     2820
           ... 
1796603       1
399298        1
5964278       1
1729613       1
128376        1
1556590       1
5517328       1
7294844       1
124282        1
1087480       1
248559        1
137940        1
453194        1
204069        1
59164         1
119469        1
85801         1
100144        1
106291        1
41603         1
391720        1
4838316       1
303619        1
1669560       1
180055        1
1065119       1
270211        1
38228         1
4109632       1
25214         1
Name: movieid, Length: 1

In [18]:
movies = df.movieid.unique().tolist()

In [19]:
df['users'] = df['userid'].astype('str')

In [20]:
users = df.userid.unique().tolist()
len(users)

1079807

In [21]:
df.dtypes

movieid            int64
userid            object
tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult            int64
startYear         object
endYear           object
runtimeMinutes    object
genres            object
users             object
dtype: object

In [22]:
# ! sudo su
! yum update -y
! yum -y install python-pip
! python -V

Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
You need to be root to perform this command.
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
You need to be root to perform this command.
Python 3.6.5 :: Anaconda, Inc.


In [23]:
! which pip

/home/ec2-user/anaconda3/envs/python3/bin/pip


In [25]:
 ! python -m pip install tqdm
# # ! python -c 'import tqdm'
 ! python -m pip install gensim
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')



# shuffle usernames
random.shuffle(users)

# extract 90% of users
users_train = [users[i] for i in range(round(0.9*len(users)))]

# split data into train and validation set
train_df = df[df['users'].isin(users_train)]
validation_df = df[~df['users'].isin(users_train)]

train_df.head()

     |████████████████████████████████| 56 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 24.2 MB 38.5 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 1.3 MB/s  eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-py3-none-any.whl size=72255 sha256=ccc904342fdafb6c85b7155912efcaf016ecc4ee0f5b0de4c664b9acb590487c
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8e/9f/cd/dbf5c1362c59abb699a218c1151679033b8ccb5b6db559d512
Successfully built smart-open


,movieid,userid,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,users
0,233469,maxmik,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller",maxmik
1,233469,rzajac,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller",rzajac
2,233469,=G=,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller",=G=
3,233469,toucanbob,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller",toucanbob
5,233469,sudipta-maji,tt0233469,movie,Collateral Damage,Collateral Damage,0,2002,\N,108,"Action,Drama,Thriller",sudipta-maji


In [ ]:
# list to capture watch history of the users
watched_train = []

# populate the list with the movie ids
for i in tqdm(users_train):
    temp = train_df[train_df["users"] == i]["movieid"].tolist()
    watched_train.append(temp)

 13%|█▎        | 122453/971826 [6:27:26<44:52:06,  5.26it/s]

In [ ]:
# list to capture purchase history of the customers
watched_val = []

# populate the list with the product codes
for i in tqdm(validation_df['users'].unique()):
    temp = validation_df[validation_df["users"] == i]["movieid"].tolist()
    watched_val.append(temp)

In [ ]:
watched_train = [[str(x) for x in lst] for lst in watched_train]

In [ ]:
watched_train[0]

In [ ]:
watched_val = [[str(x) for x in lst] for lst in watched_val]

In [ ]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(watched_train, progress_per=200)

model.train(watched_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

In [ ]:
# save word2vec model
model.save("word2vec_2.model")

In [ ]:
import gensim

model = gensim.models.Word2Vec.load("word2vec_2.model")

In [ ]:
model.init_sims(replace=True)

In [ ]:
print(model)

In [ ]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

In [ ]:
# ! sudo pip install umap-learn

In [ ]:
train_df['movieid'] = train_df['movieid'].astype(str)
train_df['movieid'][0]

In [ ]:
movies = train_df[["movieid", "primaryTitle"]]

# remove duplicates
movies.drop_duplicates(inplace=True, subset='movieid', keep="last")

# create product-ID and product-description dictionary
movies_dict = movies.groupby('movieid')['primaryTitle'].apply(list).to_dict()

In [ ]:
# test the dictionary
movies_dict['105112']



In [ ]:
def similar_movies(v, n = 6):
    
    # extract most similar movies for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract title and similarity score of the similar movies
    new_ms = []
    for j in ms:
        pair = (movies_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [ ]:
similar_movies(model['105112'])

In [ ]:
def aggregate_vectors(movies):
    movie_vec = []
    for i in movies:
        try:
            movie_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(movie_vec, axis=0)


In [ ]:
len(watched_val[2])

In [ ]:
aggregate_vectors(watched_val[2]).shape

In [ ]:
similar_movies(aggregate_vectors(watched_val[2]))

In [ ]:
similar_movies(aggregate_vectors(watched_val[2][-10:]))